In [6]:
import torch
import numpy as np
base_pth = '''/root/code/quant/bsq-net/bsq/out/resnet50_imagenet_baseline_20210908-154546/resnet50_imagenet_baseline_best.pth'''
quantized_pth = '''/root/code/quant/bsq-net/bsq/out/resnet50_imagenet_a8w8_inference_20210908-113820/resnet50_imagenet_a8w8_inference_checkpoint.pth.tar'''
qat_pth = '/root/code/quant/bsq-net/bsq/out/resnet50_imagenet_a8w8_20210910-152355/resnet50_imagenet_a8w8_best.pth'

In [7]:
base_model = torch.load(base_pth, map_location='cpu')

In [8]:
quantized_model = torch.load(quantized_pth, map_location='cpu')['state_dict']

In [9]:
qat_model = torch.load(qat_pth, map_location='cpu')

In [12]:
cnt = 0
for k, v in base_model.items():
    if v.dim() >= 1:
        cnt += np.prod(list(v.shape))
basemodel_mems = cnt * 4
print(basemodel_mems/(1024*1024))

97.69497680664062


In [13]:
cnt = 0
for k, v in quantized_model.items():
    if v.dim() >= 1:
        v = np.prod(list(v.shape))
        if 'weight' in k:
            cnt += v
        else:
            cnt += 4 * v
quantized_model_mems = cnt
print(quantized_model_mems/(1024*1024))

24.731956481933594


In [14]:
cnt = 0
for k, v in qat_model.items():
    if 'iterth' in k:
        continue
    if v.dim() >= 1:
        v = np.prod(list(v.shape))
        if 'weight' in k and 'bn' not in k:
            cnt += v
        else:
            cnt += 4 * v
qat_mems = cnt
print(qat_mems/(1024*1024))

24.824913024902344


In [15]:
quantized_model_mems / basemodel_mems

0.25315484265770855

In [16]:
qat_mems/basemodel_mems

0.2541063403294131